<center>
    <h1>Building Custom Jupyter Magics</h1>
<img src='jupyter_logo.png' style='width: 300px; padding-top: 100px'>
</center>

## Part 1: Built-In Magics

In [22]:
import numpy as np
import pandas as pd
import requests
from sklearn.model_selection import KFold
from string import ascii_lowercase

from confundo.binary import search_for_metric
import htools
import pandas_htools

In [7]:
%who

KFold	 autopep8	 htools	 json	 np	 pandas_htools	 pd	 requests	 search_for_metric	 



In [8]:
%%timeit -n 10 -r 5
arr = []
for i in range(1_000_000):
    arr.append(i)

83 ms ± 1.66 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [12]:
%%writefile spam.py
def spam(n):
    return 'spam' * n


if __name__ == '__main__':
    print(spam(5))


Writing spam.py


In [13]:
%pycat spam.py
!rm spam.py

## Part 2: Custom Magics

In [36]:
df = pd.DataFrame(np.random.randint(0, 10, (6, 10)), 
                  columns=list(ascii_lowercase)[:10])

In [37]:
df.shape
df.head(2)
df.tail(2)

,a,b,c,d,e,f,g,h,i,j
4,1,8,2,7,9,3,3,6,4,9
5,4,7,6,6,1,6,7,0,9,3


In [38]:
%%talk
df.shape
df.ends(2)

(6, 10)

,a,b,c,d,e,f,g,h,i,j
0,2,5,0,9,1,6,3,0,8,4
1,9,1,3,3,9,9,3,3,0,9
4,1,8,2,7,9,3,3,6,4,9
5,4,7,6,6,1,6,7,0,9,3


In [45]:
for i in range(5):
    kf = KFold()

/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/model_selection/_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/model_selection/_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/model_selection/_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/model_selectio

In [46]:
%%lax
for i in range(5):
    kf = KFold()

In [48]:
preds = pd.read_csv('sample_predictions.csv')
preds.head()

,y_true,y_proba
0,0.0,0.154272
1,0.0,0.000033
2,1.0,0.996420
3,0.0,0.018753
4,0.0,0.000000


In [50]:
search_for_metric('precision', .999, preds.y_true, preds.y_proba)

/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'pred

/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'pred

/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'pred

/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predic

/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/harri

...

/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'pred

/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predic

/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/harri

/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/harri

KeyboardInterrupt: 

In [52]:
%%lax
%%timebox 3
search_for_metric('precision', .999, preds.y_true, preds.y_proba)

TimeExceededError: Time limit exceeded.

In [54]:
%%lax
%%timebox 3
search_for_metric('precision', .999, preds.y_true, preds.y_proba,
                  initial_threshold=.75)

0.9984375

## Part 3:  Building Custom Magics

In [80]:
from IPython.core.magic import cell_magic, magics_class, Magics
from IPython.core.magic_arguments import (argument, magic_arguments,
                                          parse_argstring)

from htools import timebox, TimeExceededError, timebox_handler

In [61]:
@magics_class
class TimeboxMagic(Magics):

    @cell_magic
    def timebox(self, line=None, cell=None):
        print('Line: ' + line)
        print('\nCell:\n' + cell)

In [63]:
get_ipython().register_magics(TimeboxMagic)

In [64]:
%%timebox
nums = list(range(5))
print(nums)

Line: 

Cell:
nums = list(range(5))
print(nums)



In [65]:
%%timebox 123
nums = list(range(5))
print(nums)

Line: 123

Cell:
nums = list(range(5))
print(nums)



In [67]:
@magics_class
class TimeboxMagic(Magics):

    @cell_magic
    def timebox(self, line=None, cell=None):
        print('Line: ' + line)
        print('\nCell:\n' + cell)
        self.shell.run_cell(cell)

In [69]:
get_ipython().register_magics(TimeboxMagic)

In [70]:
%%timebox
nums = list(range(5))
print(nums)

Line: 

Cell:
nums = list(range(5))
print(nums)

[0, 1, 2, 3, 4]


In [71]:
@magics_class
class TimeboxMagic(Magics):

    @cell_magic
    @magic_arguments()
    @argument('time', type=int,
              help='Max number of seconds before throwing error.')
    @argument('-p', action='store_true',
              help='Boolean flag: if provided, use permissive '
                   'execution (if the cell exceeds the specified '
                   'time, no error will be thrown, meaning '
                   'following cells can still execute.) If '
                   'flag is not provided, default behavior is to '
                   'raise a TimeExceededError and halt notebook '
                   'execution.')
    def timebox(self, line=None, cell=None):
        args = parse_argstring(self.timebox, line)
        print('Line: ' + line)
        print('\nCell:\n' + cell)
        print('Args:', args)
        self.shell.run_cell(cell)

In [72]:
get_ipython().register_magics(TimeboxMagic)

In [73]:
%%timebox 5
nums = list(range(5))
print(nums)

Line: 5

Cell:
nums = list(range(5))
print(nums)

Args: Namespace(p=False, time=5)
[0, 1, 2, 3, 4]


In [81]:
%%timebox 3 -p
nums = list(range(5))
print(nums)

[0, 1, 2, 3, 4]


In [83]:
@magics_class
class TimeboxMagic(Magics):
    """Timebox a cell's execution to a user-specified duration. As with any
    standard try/except block, note that values can change during execution
    even if an error is eventually thrown (i.e. no rollback occurs).
    
    Sample usage:
    
    %%timebox 3
    # Throw error if cell takes longer than 3 seconds to execute.
    output = slow_function(*args)
    
    %%timebox 3 -p
    # Attempt to execute cell for 3 seconds, then give up. Message is printed
    # stating that time is exceeded but no error is thrown.
    output = slow_function(*args)
    """

    @cell_magic
    @magic_arguments()
    @argument('time', type=int,
              help='Max number of seconds before throwing error.')
    @argument('-p', action='store_true',
              help='Boolean flag: if provided, use permissive '
                   'execution (if the cell exceeds the specified '
                   'time, no error will be thrown, meaning '
                   'following cells can still execute.) If '
                   'flag is not provided, default behavior is to '
                   'raise a TimeExceededError and halt notebook '
                   'execution.')
    def timebox(self, line=None, cell=None):
        args = parse_argstring(self.timebox, line)
        with timebox(args.time) as tb:
            if args.p:
                cell = self._make_cell_permissive(cell)
            self.shell.run_cell(cell)

    @staticmethod
    def _make_cell_permissive(cell):
        """Place whole cell in try/except block."""
        robust_cell = (
            'try:\n\t' + cell.replace('\n', '\n\t')
            + '\nexcept:\n\tprint("Time exceeded. '
            '\\nWarning: objects may have changed during execution.")'
        )
        return robust_cell

In [84]:
get_ipython().register_magics(TimeboxMagic)

In [86]:
timebox??

In [87]:
timebox_handler??

In [104]:
%%lax
%%timebox 3

thresholds = dict()
for thresh in (.5, .75, .9, .95, .99):
    thresholds[thresh] = search_for_metric('precision', thresh, preds.y_true, preds.y_proba)

TimeExceededError: Time limit exceeded.

In [102]:
%%lax
%%timebox 6 -p

thresholds = dict()
for thresh in (.5, .75, .9, .95, .99):
    thresholds[thresh] = search_for_metric('precision', thresh, preds.y_true, preds.y_proba)

Time exceeded. 


In [103]:
print('Proceeding to next cell.')
thresholds

Proceeding to next cell.


{0.5: -1.2207031249930612e-05,
 0.75: 0.0027954101562500696,
 0.9: 0.10625000000000008,
 0.95: 0.4415283203125}

## Sources

Documentation: https://ipython.readthedocs.io/en/stable/config/custommagics.html

IPython github: https://github.com/ipython/ipython/tree/master/IPython/core

slides/notebook: https://github.com/hdmamin/custom-jupyter-magics

image source: https://jupyter.org/assets/main-logo.svg